In [18]:
import scanpy as sc
import ccAFv2

In [24]:

# Load the data from your CSV file
import scanpy as sc

# Path to the directory containing the 10x files
REH = "D:/Halima's Data/Thesis_2/1_GD428_21136_Hu_REH_Parental/outs/filtered_feature_bc_matrix/"
SUP = "D:/Halima's Data/Thesis_2/2_GD444_21136_Hu_Sup_Parental/outs/filtered_feature_bc_matrix/"
temp = SUP
adata = sc.read_10x_mtx(temp, var_names='gene_symbols', cache=True)

adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pp.filter_genes(adata, min_cells=1)  # Keeps only genes that are expressed in at least 1 cell

# Normalization and log transformation
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)



In [25]:
adata

AnnData object with n_obs × n_vars = 7728 × 27331
    obs: 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells'
    uns: 'log1p'

In [26]:
# Predict cell cycle phases
predicted_labels, probabilities = ccAFv2.predict_labels(adata, species='human', gene_id='symbol')

# Add predictions to the AnnData object
adata.obs['ccAFv2'] = predicted_labels


Running ccAFv2:
    Preparing data for classification...
    Marker genes present in this dataset: 816
    Missing marker genes in this dataset: 45
  Predicting cell cycle state probabilities...
242/242 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
  Choosing cell cycle state...
Done.


In [27]:
print(adata.obs[['ccAFv2']].head())


                     ccAFv2
AAACAGCCAATATGGA-1  Late G1
AAACAGCCACATTGCA-1  Unknown
AAACAGCCAGAATGAC-1       G1
AAACAGCCAGATAGAC-1     G2/M
AAACATGCAAGGTAAC-1  Late G1


In [28]:
# Save predictions to a CSV file

REH = "1_GD428_21136_Hu_REH_Parental"
SUP = "2_GD444_21136_Hu_Sup_Parental"
temp  = SUP

adata.obs[['ccAFv2']].to_csv(f"D:/Halima's Data/Thesis_2/RCode/Cell_Cycle_prediction_with_scATAC_Seq/paper1/Training_data/ccAFV2_{temp}.csv")
